<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 4 / SPRINT 1 / NOTE 3*

---

# N414. Hyperparameters

## 🛫 Warm Up

- [Hyperparameter Tuning Guide](https://www.youtube.com/watch?v=-i8b-srMhGM) - 딥러닝 홀로서기
- [Gradient Descent With Momentum](https://youtu.be/yWQZcdJ4k8s?t=34)
    - 학습해왔던 관성의 법칙을 유지하는 방식으로 학습 개선
- [Batch Size](https://youtu.be/U4WB9p6ODjM?t=29)
    - Batch를 크게하면 좋은 이유
    - 그러나 항상 크게할 수 없는 이유
    - 일반적으로 Batch라고 하면 Mini-batch를 의미한다는 점

- 강의자료 맨 아래, "실험 기록 프레임워크"의 Wandb [QuickStart](https://docs.wandb.com/quickstart)를 보고 회원가입 등을 해두세요

### 지난 시간 내용 복습하기

- 신경망의 순전파와 역전파 (Note 1-2)
    - 신경망의 순전파 (Note 1)
    - 신경망의 역전파 (Note 2)
    - 모델 생성과 모델 초기화 (Note 3)
    - 경사하강법의 다양성 (Note 2-3)
    - 학습 과정에서 알아야 할 Tricks (Note 3)
        - 가중치 감소/제한(Weight Decay/Constraint)
        - 드롭아웃(Dropout)
        - 학습률 계획(Learning Rate Scheduling)

- 그간 다뤄본 데이터
    - 손글씨 MNIST
    - Fashion MNIST

## 🏆 학습 목표

- 하이퍼파라미터 탐색 방법에 어떤 것이 있으며 신경망에서 조정할 수 있는 주요 하이퍼파라미터에는 어떤 것이 있는지 설명할 수 있습니다
- Scikit-learn, Keras Tuner 등을 사용하여 구축한 신경망에 하이퍼파라미터 탐색 방법을 적용할 수 있습니다.

---

## 하이퍼파라미터(Hyperparameter) 튜닝으로 성능 올리기


신경망에서는 신경써야 할 **<font color="ff6f61">하이퍼파라미터(Hyperparameter)</font>**가 굉장히 많습니다.<br/>
지금까지 다뤄온 머신러닝 알고리즘은 많아야 20개 정도의 하이퍼파라미터를 탐색하면 되었습니다.<br/>
하지만 신경망은 층이 깊어짐에 따라서 훨씬 더 조정해주어야 할 하이퍼파라미터가 많아지게 됩니다.

하이퍼파라미터 조정(Tuning)은 모델 성능에 엄청난 영향을 끼치는 요소이기 때문에 시간이 많이 소요되더라도 반드시 해주어야 합니다.<br/>
좋은 하이퍼파라미터를 찾기란 결코 쉽지 않습니다.<br/>
운좋게도 임의로 입력한 하이퍼파라미터가 만족스런 성능을 보일 수는 있지만 '기도메타'가 언제나 우리에게 성공을 보장하지는 않죠.<br/>
그렇다면 결정한 하이퍼파라미터로 구축한 모델이 좋은 성능을 보이는지를 어떻게 알 수 있을까요?

머신러닝 알고리즘을 다룰 때에 일반적인 모델의 성능을 평가하기 위해서 **<font color="ff6f61">교차 검증(Cross-Validation)</font>**을 사용하였습니다.<br/>
신경망도 역시 교차 검증을 사용하여 일반화 성능을 평가할 수 있습니다.

아래 코드를 통해 신경망에 교차 검증을 적용하는 방법에 대해 알아보겠습니다.

### 신경망으로 Boston 집값 데이터 예제 해결하기

보스턴 집값 데이터셋(`boston_housing`) 예제를 신경망으로 풀어보겠습니다.<br/>
문제를 푸는 과정에서 교차 검증을 적용하여 풀어보도록 하겠습니다!

1. **데이터셋을 불러온 후에 확인합니다.**

In [ ]:
# 데이터를 불러옵니다. 
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

65536/57026 [==================================] - 0s 0us/step


In [ ]:
print(x_train[:2])
print(y_train[:2])

[[1.23247e+00 0.00000e+00 8.14000e+00 0.00000e+00 5.38000e-01 6.14200e+00
  9.17000e+01 3.97690e+00 4.00000e+00 3.07000e+02 2.10000e+01 3.96900e+02
  1.87200e+01]
 [2.17700e-02 8.25000e+01 2.03000e+00 0.00000e+00 4.15000e-01 7.61000e+00
  1.57000e+01 6.27000e+00 2.00000e+00 3.48000e+02 1.47000e+01 3.95380e+02
  3.11000e+00]]
[15.2 42.3]


### 신경망에 교차 검증(Cross-Validation) 적용해보기

> ❗️ ***머신러닝에서 배운 교차 검증이 기억이 잘 안난다면 [링크](https://medium.com/the-owl/k-fold-cross-validation-in-keras-3ec4a3a00538)를 참조해주세요!***


2. **필요한 라이브러리를 import 합니다.**

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

3. **`KFold`를 통해 학습 데이터셋을 몇 개(k)로 나눌지를 결정합니다.**

In [ ]:
# kf 와 skf 에 각각 학습 데이터를 5개 로 나누도록 지정합니다.
kf = KFold(n_splits = 5)
skf = StratifiedKFold(n_splits = 5, random_state = 100, shuffle = True) 

x_train.shape

(404, 13)

> ❓ ***`KFold`와 `StratifiedKFold`의 차이는 무엇일지 다시 떠올려봅시다.<br/>
어떤 경우에 `KFold`가 아닌 `StratifiedKFold`를 써주어야 할까요?***

In [ ]:
y_train[:5]

array([15.2, 42.3, 50. , 21.1, 17.7])

> ❗️ ***아래부터 등장하는 코드는 고의적으로 에러를 발생하도록 쓰여 있습니다. 설명을 충분히 읽으면서 실행해 주세요!***


In [ ]:
training_data = x_train.iloc[train_index]
validation_data = x_train.iloc[val_index]

# for train_index, val_index in kf.split(np.zeros(x_train.shape[0]),y_train):
#   training_data = x_train.iloc[train_index]
#   validation_data = x_train.iloc[val_index]

AttributeError: ignored

위처럼 **Numpy 어레이(array)를 쓴다면 `.iloc` 을 쓸 수 없겠죠?**<br/>
그러니 `pd.DataFrame()` 을 이용합니다. 

In [ ]:
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)

for train_index, val_index in kf.split(np.zeros(x_train.shape[0]), y_train):
  training_data = x_train.iloc[train_index]
  validation_data = x_train.iloc[val_index]
  training_y = y_train.iloc[train_index]
  validation_y = y_train.iloc[val_index]

In [ ]:
# 데이터 확인
# training_data, validation_data, training_y, validation_y
validation_y

,0
324,19.3
325,41.3
326,20.4
327,20.5
328,13.8
...,...
399,19.4
400,25.2
401,19.4
402,19.4


아래 코드에서는 모델을 불러오는데 에러가 납니다! 무엇 때문에 나는 에러일까요?

In [ ]:
# CREATE NEW MODEL
model = Sequential()

NameError: ignored

에러명을 살펴보면 `NameError: name 'Sequential' is not defined` 입니다.<br/>
**`Sequential`이 defined 되지 않았다는 뜻이므로 해당 패키지(`Sequential`)를 import** 해주어 해결해봅시다.

In [ ]:
from tensorflow.keras.models import Sequential
# CREATE NEW MODEL
model = Sequential()

이번에는 Dense를 추가해줍니다.

In [ ]:
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

NameError: ignored

에러명 `NameError: name 'Dense' is not defined` 을 살펴보니 동일한 에러임을 알 수 있습니다.<br/>
**같은 유형의 에러이므로 같은 방법으로 해결**해보겠습니다.

In [ ]:
from tensorflow.keras.layers import Dense
# CREATE NEW MODEL
model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# COMPILE NEW MODEL
model.compile(loss='mean_squared_logarithmic_error',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(training_data, training_y,
			    epochs=2)

Epoch 1/2
11/11 [==============================] - 1s 2ms/step - loss: 0.9897 - accuracy: 0.0000e+00
Epoch 2/2
11/11 [==============================] - 0s 2ms/step - loss: 0.6488 - accuracy: 0.0000e+00


위 코드까지 모델이 잘 돌아가는 것을 확인하였습니다.

이제는 **교차 검증(Cross-Validation)을 적용할 차례**입니다.<br/>
다시 학습 데이터셋(`x_train, y_train`)을 k개 의 set으로 나누어주겠습니다. 

In [ ]:
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)

for train_index, val_index in kf.split(np.zeros(x_train.shape[0]),y_train):
  training_data = x_train.iloc[train_index, :]
  training_data_label = y_train.iloc[train_index]
  validation_data = x_train.iloc[val_index, :]
  validation_data_label = y_train.iloc[val_index]

다시 모델을 학습시켜줍니다.

In [ ]:
model.fit(training_data, training_data_label,
			epochs=10,
            batch_size=64,
			validation_data=(validation_data, validation_data_label),
          )

Epoch 1/10
6/6 [==============================] - 0s 56ms/step - loss: 0.3301 - accuracy: 0.0000e+00 - val_loss: 0.1409 - val_accuracy: 0.0000e+00
Epoch 2/10
6/6 [==============================] - 0s 7ms/step - loss: 0.1566 - accuracy: 0.0000e+00 - val_loss: 0.1735 - val_accuracy: 0.0000e+00
Epoch 3/10
6/6 [==============================] - 0s 6ms/step - loss: 0.1519 - accuracy: 0.0000e+00 - val_loss: 0.1215 - val_accuracy: 0.0000e+00
Epoch 4/10
6/6 [==============================] - 0s 6ms/step - loss: 0.1103 - accuracy: 0.0000e+00 - val_loss: 0.1052 - val_accuracy: 0.0000e+00
Epoch 5/10
6/6 [==============================] - 0s 6ms/step - loss: 0.1109 - accuracy: 0.0000e+00 - val_loss: 0.1041 - val_accuracy: 0.0000e+00
Epoch 6/10
6/6 [==============================] - 0s 6ms/step - loss: 0.1010 - accuracy: 0.0000e+00 - val_loss: 0.0901 - val_accuracy: 0.0000e+00
Epoch 7/10
6/6 [==============================] - 0s 6ms/step - loss: 0.0910 - accuracy: 0.0000e+00 - val_loss: 0.0887 - va

데이터가 잘 나누어져 들어갔는 지 확인을 해봅니다. 

In [ ]:
print(training_data[:2])
print(training_data.shape)

        0     1     2    3      4   ...   8      9     10      11     12
0  1.23247   0.0  8.14  0.0  0.538  ...  4.0  307.0  21.0  396.90  18.72
1  0.02177  82.5  2.03  0.0  0.415  ...  2.0  348.0  14.7  395.38   3.11

[2 rows x 13 columns]
(324, 13)


In [ ]:
training_data_label[:2]

,0
0,15.2
1,42.3


In [ ]:
# COMPILE NEW MODEL
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(training_data, training_data_label,
		    epochs=10,
            batch_size=30,
          )

Epoch 1/10
11/11 [==============================] - 0s 3ms/step - loss: 76.0882
Epoch 2/10
11/11 [==============================] - 0s 2ms/step - loss: 79.1451
Epoch 3/10
11/11 [==============================] - 0s 2ms/step - loss: 65.2776
Epoch 4/10
11/11 [==============================] - 0s 2ms/step - loss: 56.1669
Epoch 5/10
11/11 [==============================] - 0s 3ms/step - loss: 52.3630
Epoch 6/10
11/11 [==============================] - 0s 2ms/step - loss: 50.8348
Epoch 7/10
11/11 [==============================] - 0s 2ms/step - loss: 51.6367
Epoch 8/10
11/11 [==============================] - 0s 3ms/step - loss: 50.1368
Epoch 9/10
11/11 [==============================] - 0s 2ms/step - loss: 51.3770
Epoch 10/10
11/11 [==============================] - 0s 2ms/step - loss: 45.9958


In [ ]:
# COMPILE NEW MODEL
# 다양한 loss로 테스트도 해봅니다. 
model.compile(loss='binary_crossentropy', optimizer='adam') # binary_crossentropy # mean_squared_error
model.fit(x_train, y_train,
			epochs=10,
            batch_size=30,
            validation_data = (validation_data, validation_data_label),
          )

Epoch 1/10
14/14 [==============================] - 1s 15ms/step - loss: -326.2582 - val_loss: -345.8718
Epoch 2/10
14/14 [==============================] - 0s 3ms/step - loss: -326.2582 - val_loss: -345.8718
Epoch 3/10
14/14 [==============================] - 0s 3ms/step - loss: -326.2582 - val_loss: -345.8718
Epoch 4/10
14/14 [==============================] - 0s 4ms/step - loss: -326.2582 - val_loss: -345.8718
Epoch 5/10
14/14 [==============================] - 0s 3ms/step - loss: -326.2582 - val_loss: -345.8718
Epoch 6/10
14/14 [==============================] - 0s 3ms/step - loss: -326.2582 - val_loss: -345.8718
Epoch 7/10
14/14 [==============================] - 0s 4ms/step - loss: -326.2582 - val_loss: -345.8718
Epoch 8/10
14/14 [==============================] - 0s 4ms/step - loss: -326.2582 - val_loss: -345.8718
Epoch 9/10
14/14 [==============================] - 0s 4ms/step - loss: -326.2582 - val_loss: -345.8718
Epoch 10/10
14/14 [==============================] - 0s 3ms/ste

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test mse:", results)

1/1 [==============================] - 0s 427ms/step - loss: -336.6792
test loss, test mse: -336.6792297363281


이제 한번에 테스트를 수행해봅니다. 

In [ ]:
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)
for train_index, val_index in kf.split(np.zeros(x_train.shape[0])):
  training_data = x_train.iloc[train_index, :]
  training_data_label = y_train.iloc[train_index]
  validation_data = x_train.iloc[val_index, :]
  validation_data_label = y_train.iloc[val_index]

  # CV
  model.compile(loss='mean_squared_error', optimizer='adam')
  model.fit(x_train, y_train,
			    epochs=10,
          batch_size=30,
          validation_data = (validation_data, validation_data_label),
          )
  results = model.evaluate(x_test, y_test, batch_size=128)
  print("test loss, test mse:", results)

Epoch 1/10
14/14 [==============================] - 0s 13ms/step - loss: 90.6808 - val_loss: 133.9628
Epoch 2/10
14/14 [==============================] - 0s 3ms/step - loss: 82.7211 - val_loss: 76.0921
Epoch 3/10
14/14 [==============================] - 0s 4ms/step - loss: 50.4193 - val_loss: 45.0233
Epoch 4/10
14/14 [==============================] - 0s 3ms/step - loss: 46.3767 - val_loss: 39.1982
Epoch 5/10
14/14 [==============================] - 0s 4ms/step - loss: 46.6952 - val_loss: 46.5693
Epoch 6/10
14/14 [==============================] - 0s 4ms/step - loss: 45.5071 - val_loss: 46.4023
Epoch 7/10
14/14 [==============================] - 0s 3ms/step - loss: 49.6951 - val_loss: 41.6469
Epoch 8/10
14/14 [==============================] - 0s 3ms/step - loss: 44.0159 - val_loss: 37.5868
Epoch 9/10
14/14 [==============================] - 0s 3ms/step - loss: 44.8145 - val_loss: 38.7562
Epoch 10/10
1/1 [==============================] - 0s 97ms/step - loss: 53.4831
test loss, test ms

이렇게 하면, CV를 통해서 모델을 돌릴 수 있는 것까지 확인해보았습니다. 

### 입력 데이터 정규화 (Normalizing)

입력 데이터 정규화(Normalizing, Scaling)에 대해서 복습해보겠습니다.

신경망에서는 입력 데이터 정규화가 무조건 필요하지는 않습니다.<br/>
신경망이 수치형 데이터를 받으면 자체적으로 적절한 가중치를 학습하기 때문인데요.

하지만 정규화를 해주면 **학습을 빠르게 해주고, 최적화 과정에서 지역 최적점(Local optimum)에 빠질 위험을 줄여줍니다.**<br/>
그렇기 때문에 가능하다면 정규화를 해주는 것이 좋겠죠?

> ❗️ ***해당 내용이 담겨 있는 Stackoverflow [링크](https://stackoverflow.com/questions/4674623/why-do-we-have-to-normalize-the-input-for-an-artificial-neural-network)입니다. 시간이 나면 읽어보도록 합시다.***

In [ ]:
# 정규화를 위한 함수 호출
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
print(x_train[:10])

[[-0.27224633 -0.48361547 -0.43576161 -0.25683275 -0.1652266  -0.1764426
   0.81306188  0.1166983  -0.62624905 -0.59517003  1.14850044  0.44807713
   0.8252202 ]
 [-0.40342651  2.99178419 -1.33391162 -0.25683275 -1.21518188  1.89434613
  -1.91036058  1.24758524 -0.85646254 -0.34843254 -1.71818909  0.43190599
  -1.32920239]
 [ 0.1249402  -0.48361547  1.0283258  -0.25683275  0.62864202 -1.82968811
   1.11048828 -1.18743907  1.67588577  1.5652875   0.78447637  0.22061726
  -1.30850006]
 [-0.40149354 -0.48361547 -0.86940196 -0.25683275 -0.3615597  -0.3245576
  -1.23667187  1.10717989 -0.51114231 -1.094663    0.78447637  0.44807713
  -0.65292624]
 [-0.0056343  -0.48361547  1.0283258  -0.25683275  1.32861221  0.15364225
   0.69480801 -0.57857203  1.67588577  1.5652875   0.78447637  0.3898823
   0.26349695]
 [-0.37502238 -0.48361547 -0.54747912 -0.25683275 -0.54935658 -0.78865126
   0.18954148  0.48371503 -0.51114231 -0.71552978  0.51145832  0.38669063
  -0.13812828]
 [ 0.58963463 -0.48361547

### 모델 성능을 자동 검증(Validation) 하는 기능을 사용해봅시다.

이전에는 검증 데이터셋을 나누기 위해서 `sklearn` 의 `train_test_split` 패키지를 사용하였는데요.<br/>
케라스에는 보다 쉽게 검증을 할 수 있는 `validation_data`라는 편리한 기능이 있습니다.<br/>
모델 학습 시 `validation_data`에 검증 데이터를 입력하면 케라스에서 자동으로 검증용 데이터로 사용하여 성능을 측정합니다.

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 중요한 하이퍼 파라미터들
inputs = x_train.shape[1]
epochs = 75                 # 전체 반복횟수
batch_size = 10             # 한번에 학습하는 사이즈


# 모델을 생성합니다
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Sequential인 경우, 아래의 방법으로도 모델을 만들 수 있습니다.
# model = Sequential(
# [
#     Dense(64, activation='relu', input_shape=(inputs,)),
#     Dense(64, activation='relu'),
#     Dense(1)
# ]
# )

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# Fit Model
model.fit(x_train, y_train, 
          validation_data=(x_test,y_test),  # validation set
          epochs=epochs,                    # 전체 반복횟수
          batch_size=batch_size             # 한번에 학습하는 사이즈
         )

Epoch 1/75
41/41 [==============================] - 1s 6ms/step - loss: 513.8185 - mse: 513.8185 - mae: 20.5771 - val_loss: 440.6191 - val_mse: 440.6191 - val_mae: 18.8570
Epoch 2/75
41/41 [==============================] - 0s 3ms/step - loss: 263.8185 - mse: 263.8185 - mae: 13.5170 - val_loss: 114.4852 - val_mse: 114.4852 - val_mae: 8.8211
Epoch 3/75
41/41 [==============================] - 0s 3ms/step - loss: 64.3071 - mse: 64.3071 - mae: 5.7884 - val_loss: 49.5342 - val_mse: 49.5342 - val_mae: 5.3628
Epoch 4/75
41/41 [==============================] - 0s 3ms/step - loss: 38.1561 - mse: 38.1561 - mae: 4.4052 - val_loss: 36.3478 - val_mse: 36.3478 - val_mae: 4.7159
Epoch 5/75
41/41 [==============================] - 0s 3ms/step - loss: 29.1752 - mse: 29.1752 - mae: 3.7983 - val_loss: 30.4761 - val_mse: 30.4761 - val_mae: 4.4106
Epoch 6/75
41/41 [==============================] - 0s 3ms/step - loss: 24.8251 - mse: 24.8251 - mae: 3.5279 - val_loss: 28.8393 - val_mse: 28.8393 - val_mae: 

## 신경망에서의 하이퍼 파라미터 튜닝

### 하이퍼파라미터 튜닝 방식의 종류

1. **"Babysitting"(육아) 혹은 "Grad Student Descent"(대학원생 갈아넣기)**

    다윈의 진화론을 아시나요? 진화론에서는 '자연 선택'이란 단어가 진화를 주도했다고 말하곤 합니다.<br/>
하지만 하이퍼 파라미터 선택은 자연이 해주지 않습니다. 그렇다면 우리가 직접 하는 수 밖에 없겠죠?<br/>
이전 프로젝트나 이번 스프린트에서 모델 성능을 높이기 위해 여러 숫자를 직접 넣어보며 하이퍼 파라미터를 수없이 조정했다면,<br/>
첫 번째 방법을 수행했다고 말할 수 있겠습니다.

    100% **<font color="ff6f61">수작업(Manual)</font>**으로 파라미터를 수정하는 방법입니다.<br/>
학계에서 논문을 출간할 수 있을 정도로 놀라운 정확도를 보여주는 하이퍼파라미터의 수치를 찾아내기 위해 쓰는 방법이죠.<br/>
이를 위해서 실험자의 경험이나 도메인 지식이 필요하기도 합니다.<br/>
~~*(물론 지도교수님들이 이 걸 직접 하시진 않습니다, 교수님의 시간은 소중하니까요...)*~~

2. **Grid Search**

    하지만 언제까지나 이렇게 하나하나 수작업으로만 시도해 볼 수는 없겠죠.<br/>
1번 방식을 자동화한 방법이 바로 **<font color="ff6f61">"Grid Search"</font>**입니다.<br/>
이 방법에서는 하이퍼파라미터마다 탐색할 지점을 정해주면 모든 지점에 해당하는 조합을 알아서 수행합니다.

    Grid Search는 학습을 실행한 뒤 한참 놀다오면 되는 매우 편한 방법이지만 **장점만 있는 것은 아닙니다.**<br/>
범위를 너무 많이 설정하면 '좀 놀다 오면 끝나는' 수준을 넘어 '수료하고 취직을 하고 나서도 끝나지 않을 수도' 있는데요.<br/>
만약 5개의 파라미터에 대해 각각 5개의 지점을 지정해주면 Grid Search는 총 $5^5=3,125$ 번의 모델 학습을 진행하게 됩니다.<br/>
여기에 5번의 교차 검증까지 진행한다면 모델은 $3,125 \times 5 = 15,625$ 번이나 학습을 수행합니다.<br/>
모델 한 번 학습에 10분만 걸린다고 쳐도 **3달 반**이 걸리는 무시무시한 작업입니다. 실제로 이런 일은 없어야겠죠?

    그렇기 때문에 Grid Search 로 너무 많은 하이퍼파라미터 조합을 찾으려고 하지 않는 것이 좋습니다.<br/>
1개, 혹은 최대 2개 정도의 파라미터 최적값을 찾는 용도로 적합합니다.<br/>
굳이 많은 하이퍼파라미터 조합을 시도할 필요는 없습니다.<br/>
모델 성능에 **보다 직접적인 영향을 주는 하이퍼파라미터가 따로 있기 때문**인데요.<br/>
이러한 파라미터만 제대로 튜닝해서 최적값을 찾은 후 나머지 하이퍼파라미터도 조정해나가면 못해도 90% 이상의 성능을 확보할 수 있습니다.<br/>
이런 식으로 하나씩 접근하다 보면 적어도 무한루프가 발생하는 위험은 줄일 수 있습니다.

3. **Random Search**

    **<font color="ff6f61">"Random Search"</font>** 는 무한 루프라는 Grid Search의 단점을 해결하기 위해 나온 방법입니다.<br/>
Random Search 는 지정된 범위 내에서 무작위로 모델을 돌려본 후 최고 성능의 모델을 반환합니다.<br/> 시도 횟수를 정해줄 수 있기 때문에 Grid Search 에 비해서 훨씬 적은 횟수로도 끝마칠 수 있겠죠?

    Grid Search 에서는 파라미터의 중요도가 모두 동등하다고 가정합니다.<br/>
하지만 위에서 알아본 것처럼 실제로 더 중요한 하이퍼파라미터가 있는데요.<br/>
Random Search 는 **상대적으로 중요한 하이퍼파라미터에 대해서는 탐색을 더 하고, 덜 중요한 하이퍼파라미터에 대해서는 실험을 덜 하도록** 합니다.

    Random Search 는 절대적으로 완벽한 하이퍼파라미터를 찾아주지는 않는다는 단점을 가지고 있는데요.<br/>
하지만 Grid Search와 비교했을 때, 학습에 걸리는 시간이 훨씬 더 적다는 점으로도 Random Search의 의의를 찾을 수 있습니다.

> ❗️ ***아래 그림을 보면서 Grid Search 와 Random Search 의 차이에 대해서 생각해봅시다.***

<img src="https://i.imgur.com/qwySX8w.png" width="600">

4. **Bayesian Methods**

    "Baby sitting" 이나 "Grid Search" 등의 방식에서는 탐색 결과를 보고, 결과 정보를 다시 새로운 탐색에 반영하면 성능을 더 높일 수 있었습니다.<br/> **<font color="ff6f61">베이지안 방식(Bayesian Method)</font> 은 이렇게 이전 탐색 결과 정보를 새로운 탐색에 활용하는 방법**입니다.<br/>
그렇기 때문에 베이지안 방법을 사용하면 하이퍼파라미터 탐색 효율을 높일 수 있습니다.<br/>
`bayes_opt` 나 `hyperopt`와 같은 패키지를 사용하면 베이지안 방식을 적용할 수 있습니다.

### 튜닝 가능한 파라미터에는 어떤 것이 있을까요?

탐색해 볼 수 있는 하이퍼파라미터의 종류는 다음과 같습니다.

- 배치 크기(batch_size)
- 반복 학습 횟수(에포크, training epochs)
- 옵티마이저(optimizer)
- 학습률(learning rate)
- 활성화 함수(activation functions)
- Regularization(weight decay, dropout 등)
- 은닉층(Hidden layer)의 노드(Node) 수

> ❗️ ***실제로는 이보다 더 많은 하이퍼파라미터를 튜닝할 수 있습니다.<br/>
하지만 일단은 이정도만 기억해도 좋습니다. 반복하여 시도하다 보면 익숙해질 것입니다.***

In [ ]:
# 데이터를 불러옵니다. 
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

# 정규화를 위한 함수 호출
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
print(x_train[:10])

[[-0.27224633 -0.48361547 -0.43576161 -0.25683275 -0.1652266  -0.1764426
   0.81306188  0.1166983  -0.62624905 -0.59517003  1.14850044  0.44807713
   0.8252202 ]
 [-0.40342651  2.99178419 -1.33391162 -0.25683275 -1.21518188  1.89434613
  -1.91036058  1.24758524 -0.85646254 -0.34843254 -1.71818909  0.43190599
  -1.32920239]
 [ 0.1249402  -0.48361547  1.0283258  -0.25683275  0.62864202 -1.82968811
   1.11048828 -1.18743907  1.67588577  1.5652875   0.78447637  0.22061726
  -1.30850006]
 [-0.40149354 -0.48361547 -0.86940196 -0.25683275 -0.3615597  -0.3245576
  -1.23667187  1.10717989 -0.51114231 -1.094663    0.78447637  0.44807713
  -0.65292624]
 [-0.0056343  -0.48361547  1.0283258  -0.25683275  1.32861221  0.15364225
   0.69480801 -0.57857203  1.67588577  1.5652875   0.78447637  0.3898823
   0.26349695]
 [-0.37502238 -0.48361547 -0.54747912 -0.25683275 -0.54935658 -0.78865126
   0.18954148  0.48371503 -0.51114231 -0.71552978  0.51145832  0.38669063
  -0.13812828]
 [ 0.58963463 -0.48361547

#### 배치 사이즈(Batch Size)

**<font color="ff6f61">배치 사이즈(Batch size)</font>**는 순전파/역전파를 통해 모델의 가중치를 업데이트 할 때마다,<br/>
즉 매 iteration 마다 **몇 개의 입력 데이터를 볼지를** 결정하는 하이퍼파라미터입니다.<br/>
최적의 배치 사이즈를 찾는 과정은 왜 중요할까요?

**배치 사이즈를 너무 크게 하면** 한 번에 많은 데이터에 대한 Loss를 계산해야 한다는 단점이 생깁니다.<br/>
이럴 경우 가중치 업데이트가 빠르게 이루어지지 않는데다, 주어진 Epoch 안에 충분한 횟수의 iteration을 확보할 수 없게 됩니다.<br/>
그리고 파라미터가 굉장히 많은 모델에 큰 배치 사이즈를 적용하게 될 경우 메모리를 초과해버리는 현상(Out-of-Memory)이 발생하기도 합니다.<br/>
반대로 **배치 사이즈를 너무 작게 설정하면** 학습에 오랜 시간이 걸리고, 노이즈가 많이 발생한다는 단점도 있습니다.

일반적으로 배치 사이즈는 $32-512$ 사이의 2의 제곱수로 결정하여 줍니다.<br/>
케라스 배치 사이즈의 기본값(Default)은 $32$ 로 설정되어 있습니다.

> ❗️ ***아래의 글은 나중에 학습하면서 보아도 좋습니다. 일단은 건너뛰고 추후 학습시에 참조하도록 합시다.<br/>
이미지 처리에서 작은 배치 사이즈($<32$)를 잘 설정하면 일반화(Generalization) 성능을 높일 수 있다는 내용의 [논문](https://arxiv.org/abs/1804.07612)입니다.<br/>
배치 사이즈를 왜 2의 제곱수로 설정하는지 궁금하다면 다음 Stackoverflow [링크](https://datascience.stackexchange.com/questions/20179/what-is-the-advantage-of-keeping-batch-size-a-power-of-2)를 참조해봅시다.***

배치 사이즈를 조정하여 최적의 배치 사이즈를 찾아보겠습니다.

1. **필요한 패키지를 import 합니다.**

In [ ]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

2. **재현성을 위해 랜덤시드를 고정합니다**

In [ ]:
numpy.random.seed(1100)

3. **데이터셋을 불러온 후에 Feature 와 Label로 분리합니다.**

In [ ]:
# 데이터셋을 불러옵니다.
url ="https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

dataset = pd.read_csv(url, header=None).values

In [ ]:
# 불러온 데이터셋을 X와 Y로 나눕니다
X = dataset[:,0:8]
Y = dataset[:,8]

4. **모델을 제작합니다.**

`KerasClassifier` 로 wrapping 하기 위하여 함수로 정의합니다. 

In [ ]:
def create_model():
    # 모델 제작
    model = Sequential()
    model.add(Dense(100, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # 모델 컴파일링
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

4. **`KerasClassifier` 로 wrapping 하여줍니다.**

In [ ]:
# keras.wrapper를 활용하여 분류기를 만듭니다
model = KerasClassifier(build_fn=create_model, verbose=0)

In [ ]:
# GridSearch
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [30]
param_grid = dict(batch_size=batch_size)

5. **하이퍼파라미터 탐색을 위한 `GridSearchCV` 를 설정하고 학습합니다.**

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

6. **최적의 결과를 낸 하이퍼파라미터와 각각의 결과를 출력해봅시다.**

In [ ]:
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.5793650805950165 using {'batch_size': 80}
Means: 0.5728715777397155, Stdev: 0.056264070333286655 with: {'batch_size': 10}
Means: 0.5586028277873993, Stdev: 0.06420384197886535 with: {'batch_size': 20}
Means: 0.5612596690654754, Stdev: 0.07808589018263502 with: {'batch_size': 40}
Means: 0.566581791639328, Stdev: 0.08425900074864029 with: {'batch_size': 60}
Means: 0.5793650805950165, Stdev: 0.061918396158844716 with: {'batch_size': 80}
Means: 0.4463627874851227, Stdev: 0.11636758131443188 with: {'batch_size': 100}


#### 옵티마이저(Optimizer)

**<font color="ff6f61">옵티마이저(Optimizer)</font>** 역시 굉장히 중요한 하이퍼파라미터입니다.<br/>
`adam` 이라는 옵티마이저가 꽤 좋은 성능을 보장하기 때문에 많이 사용됩니다.<br/>
최근에는 `adam` 을 개선하거나 기능을 추가한 `adamW, adamP`와 같은 옵티마이저도 사용되죠.<br/>
중요한 점은 **"모든 경우에 좋은 옵티마이저란 없다"**는 것인데요.<br/>
그렇기 때문에 모델에 따라, 데이터셋에 따라 적절한 옵티마이저를 잘 설정해주어야 합니다.

게다가 어떤 옵티마이저를 선택하는 지에 따라서 최적의 하이퍼파라미터 값이 달라집니다.<br/>
그렇기 때문에 옵티마이저를 다르게 해줄 때마다 적절한 학습률(`learning rate`)과 모멘텀(`momentum`) 등을 다르게 설정해주는 것이 좋습니다.

> ❗️ ***여러 가지 옵티마이저가 잘 정리된 [블로그 글](https://sacko.tistory.com/42)도 읽어보도록 합시다.***


#### 학습률(Learning Rate)

**<font color="ff6f61">학습률(Learning rate, `lr`)</font>** 은 옵티마이저에서 지정해 줄 수 있는 하이퍼파라미터 중 하나입니다.

Keras에서 학습률의 기본값은 어떻게 설정되어 있을까요?<br/>
[링크](https://www.google.com/search?q=keras+default+learning+rate&oq=keras+default+learning+rate&aqs=chrome..69i57j0i22i30l2.4191j0j7&sourceid=chrome&ie=UTF-8)에서 볼 수 있듯 케라스의 기본 학습률은 0.001로 설정되어 있습니다.

학습률이 너무 높으면 경사 하강 과정에서 발산하면서 모델이 최적값을 찾을 수 없게 되어버립니다.<br/>
반대로 너무 낮게 설정할 경우에는 최적점에 이르기까지 너무 오래 걸리거나, 주어진 iteration 내에서 모델이 수렴하는데 실패하기도 합니다.<br/>
그렇기 때문에 최적의 학습률을 찾는 것은 학습에서 중요한 요소입니다.<br/>

> ❗️ ***아래는 학습률이 너무 클 때와 작을 때의 경사하강법을 나타낸 그림입니다.<br/>
그림을 기억하면서 최적의 학습률이 왜 중요한 지에 대해 생각해봅시다.***

<img src="https://i.imgur.com/RfBFgKs.png" width="600">

처음에는 $[0.001, 0.01, 0.1, 0.2, 0.3, 0.5]$ 정도로 넓은 범위에서 크기 순으로 학습률을 튜닝해봅니다.<br/>
학습률을 0.5 정도로 높게 잡는 것은 추천하지 않지만, 이 때의 결과를 분석하며 과하게 높은 학습률이 왜 좋지 않은지를 경험해보는 것도 좋은 공부가 됩니다.

위 과정을 통해 최적의 학습률 값을 찾았다면 해당 학습률 주변에서 다시 최적의 학습률 값을 찾아봅시다.<br/>
만약 위에서 학습률 값이 0.1일 때, 가장 좋은 성능을 보였다면 $[0.05, 0.08, 0.1, 0.12, 0.15]$ 정도로 시도해보면 좋습니다.

학습률을 조정하면 최적값에 도달할 수 있는 iteration의 횟수 역시 변경됩니다.<br/>
그렇기 때문에 학습률을 튜닝할 때에는 Epoch 의 횟수도 함께 튜닝해주는 것이 좋습니다.

#### 모멘텀(Momentum)


**<font color="ff6f61">모멘텀(Momentum)</font>**은 옵티마이저에 관성을 부여하는 하이퍼파라미터입니다.<br/>
모멘텀은 이전 iteration에서 경사 하강을 한 정도를 새로운 iteration에 반영합니다.<br/>
그렇기 때문에 아래 그림에 등장하는 **지역 최저점(Local minima)에 빠지지 않을 수 있도록 합니다.**

<img src="https://i.imgur.com/1F0NNID.png" width="400">

#### 가중치 초기화(Network Weight Initialization)

초기 가중치를 어떻게 설정할 지를 결정하는 **<font color="ff6f61">가중치 초기화(Weight initialization)</font>**는 신경망에서 매우 중요한 요소입니다.

신경망의 가중치를 초기화 하는 방법은 여러 가지가 있습니다.<br/>
케라스에서는 아래와 같은 가중치 초기화 방법을 제공하고 있습니다.

```python
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
```

아래에서는 먼저 가중치를 정규분포로 초기화하였을 때의 문제를 알아보고<br/>
흔히 사용되는 **Xavier(=`glorot`) 초기화**와 **He(=`he`) 초기화**에 대해 알아보도록 하겠습니다.

> ❗️ ***아래 가중치 초기화 방법의 수식을 외울 필요는 없습니다.<br/>
일단은 어떤 초기화 방법이 있고 해당 초기화 방법이 언제 사용되는지만 기억하고 넘어갑시다.***

1. **표준편차를 1인 정규분포로 가중치를 초기화 할 때 각 층의 활성화 값 분포**

표준편차가 일정한 정규분포로 가중치를 초기화 해 줄 때에는 대부분의 활성화 값이 0과 1에 위치하는 것을 볼 수 있습니다.<br/>
활성값이 고르지 못할 경우 학습이 제대로 이루어지지 않는데요.<br/>
그렇기 때문에 가장 간단한 방법임에도 잘 사용되지 않습니다.


<img src="https://t1.daumcdn.net/cfile/tistory/994C2F3C5AB623C526" width=500/>

2. **Xavier 초기화를 해주었을 때의 활성화 값의 분포**

**Xavier 초기화(Xavier initialization)**는 가중치를 표준편차가 고정값인 정규분포로 초기화 했을 때의 문제점을 해결하기 위해 등장한 방법입니다.<br/>
Xavier 초기화는 이전 층의 노드가 $n$ 개일 때, 현재 층의 가중치를 표준편차가 $\frac{1}{\sqrt{n}}$ 인 정규분포로 초기화합니다.<br/>
*(케라스에서 Xavier 초기화는 이전 층의 노드가 $n$ 개이고 현재 층의 노드가 $m$ 개일 때, 현재 층의 가중치를 표준편차가 $\frac{2}{\sqrt{n+m}}$ 인 정규분포로 초기화합니다.)*

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbuwPPz%2FbtquO7Wq9Rp%2Fylz2Qsc0fi9m0TaQNXBYDK%2Fimg.png" width=500/>

3. **He 초기화를 해주었을 때의 활성화 값의 분포**

Xavier 초기화는 활성화 함수가 시그모이드(`Sigmoid`)인 신경망에서는 잘 동작합니다.<br/>
하지만 활성화 함수가 ReLU 일 경우에는 층이 지날수록 활성값이 고르지 못하게 되는 문제를 보이는데요.

이런 문제를 해결하기 위해 등장한 것이 바로 **He 초기화(He initialization)** 입니다.<br/>
He 초기화는 이전 층의 노드가 $n$ 개일 때, 현재 층의 가중치를 표준편차가 $\frac{2}{\sqrt{n}}$ 인 정규분포로 초기화합니다.<br/>
He 초기화를 적용하면 아래 그림처럼 층이 지나도 활성값이 고르게 유지되는 것을 확인할 수 있습니다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcKBoWH%2FbtquO7B8MfF%2FMs5LyROpCV89EbCFNXja4k%2Fimg.png" width=500/>

가중치 초기화 방법을 요약하면 다음과 같습니다.

**c.f. Activation function에 따른 초기값 추천**
1. Sigmoid  ⇒  Xavier 초기화를 사용하는 것이 유리 
2. ReLU  ⇒  He 초기화 사용하는 것이 유리

#### (복습) 활성화 함수(Activation Function)

은닉층의 활성화 함수 역시 조정해 볼 수 있는 하이퍼파라미터 중 하나입니다.<br/>
지난 몇 시간에 걸쳐 이미 몇 가지 활성화 함수를 소개드렸는데요.<br/>
보통은 **은닉층에는 ReLU를, 출력층에는 문제에 따라 Sigmoid (이진 분류)나 Softmax(다중 분류)를 적용**하는 것을 볼 수 있었습니다.

하지만 모델에 따라서 은닉층에도 시그모이드(`sigmoid`)나 Hyperbolic tangent(`tanh`) 등의 다른 활성화 함수를 적용할 수 있습니다.<br/>
여러 경우를 시도해보고 결과가 더 괜찮게 나오는지 확인해 보도록 합시다.

#### (복습) Regularization(weight decay, dropout 등)

과적합(Overfitting)을 방지하기 위한 Regularization을 적용한다면 이 또한 중요한 하이퍼파라미터입니다.

**가중치 감소(Weight decay)를 얼마나 적용**할 것인지 혹은 **가중치 제한(Weight constraint)의 범위를 어떻게 설정**할 것인지에 따라 신경망의 성능이 결정됩니다.

드롭아웃 값은 **매 iteration 마다 임의로 비활성화 하고 싶은 뉴런의 비율**입니다.<br/>
다음 주에 등장하는 다양한 모델에서는 드롭아웃이 많이 적용되어 있는 것을 보실 수 있습니다.<br/>
그 때, '과적합을 해결하기 위해서 드롭아웃을 적용한 것이구나'를 기억하실 수 있으면 좋겠습니다.

#### (복습) 은닉층 노드 수

일반적으로 은닉층의 노드 수를 늘림으로써 모델을 복잡하게 만들어 줄 수록 데이터의 복잡한 패턴을 잘 이해할 수 있습니다.<br/>
하지만 노드가 많아지고 층이 깊어질수록 학습 시간이 길어지고 과적합에 대한 위험이 늘어나게 되는데요.<br/>
그렇기 때문에 **각 층의 노드 수를 잘 조정하는 것 역시 딥러닝의 성능을 높이기 위해서 중요한 요소**
입니다.

## 라이브러리를 사용한 하이퍼파라미터 튜닝

### Keras Tuner 를 사용하여 Fashion MNIST 예제를 위한 최적의 하이퍼파라미터 탐색하기


**<font color="ff6f61">Keras Tuner</font>** 는 케라스 프레임워크에서 하이퍼파라미터를 튜닝하는 데 도움이 되는 라이브러리입니다.<br/>
Fashion MNIST 예제에 Keras Tuner를 적용하여 하이퍼파라미터 튜닝을 수행해보겠습니다.

1. **필요한 패키지를 import 합니다.**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten
import IPython

2. **Keras Tuner를 설치한 후 import 합니다.**

Keras Tuner는 Colab에 내장된 패키지가 아니기 때문에 따로 설치를 해준 후에 import 하여줍니다.

In [ ]:
!pip install -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 97 kB 2.8 MB/s 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


3. **데이터셋을 불러온 후에 정규화(Normalizing) 해줍니다.**

Fashion MNIST 데이터셋을 불러온 후에 이미지를 0-1 사이의 값으로 정규화합니다.

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

4. **Model을 제작합니다.**

모델을 제작하고 탐색할 하이퍼파라미터 범위와 지점을 정의합니다.<br/>

아래 단계에서 2가지 방법을 적용해 볼 수 있는데요.
1. Model builder function(`model_builder`)을 사용
2. Keras Tuner API의 `HyperModel` 클래스에 있는 분류기를 사용 ▶️ [링크](https://keras.io/api/keras_tuner/hypermodels/base_hypermodel/)를 참조해봅시다.

아래에서는 1번에 해당하는 방법을 수행해보겠습니다.<br/>
`model_builder` 라는 함수를 정의하고 해당 함수 내부에서 모델 설계와 하이퍼파라미터 튜닝까지 모두 수행할 수 있도록 하였습니다.

> ❗️ ***심화 학습 : 나중에 보도록 합시다.<br/>
이미지 처리를 위한 몇 가지 모델에서는 HyperModel의 하위 클래스인 [HyperXception](https://keras.io/api/keras_tuner/hypermodels/hyper_xception/#hyperxception-class) 및 [HyperResNet](https://keras.io/api/keras_tuner/hypermodels/hyper_resnet/#hyperresnet-class) 등을 적용할 수 있습니다.***

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(Flatten(input_shape=(28, 28)))
  
  # 첫 번째 Dense layer에서 노드 수를 조정(32-512)합니다.
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(Dense(units = hp_units, activation = 'relu'))
  model.add(Dense(10, activation='softmax'))

  # Optimizer의 학습률(learning rate)을 조정[0.01, 0.001, 0.0001]합니다. 
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

5. **하이퍼파라미터 튜닝을 수행할 튜너(Tuner)를 지정합니다.**

Keras Tuner 에서는 **Random Search, Bayesian Optimization, Hyperband** 등의 최적화 방법을 수행할 수 있습니다.<br/>
아래에서는 `Hyperband` 를 통해서 튜닝을 수행해보도록 하겠습니다.

Hyperband 사용 시 Model builder function(`model_builder`), 훈련할 최대 epochs 수(`max_epochs`) 등을 지정해주어야 합니다.<br/>
Hyperband 는 리소스를 알아서 조절하고 조기 종료(Early-stopping) 기능을 사용하여 
높은 성능을 보이는 조합을 신속하게 통합한다는 장점을 가지고 있습니다.



In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')                       

6. **Callback 함수를 지정합니다.**

하이퍼파라미터 탐색을 실행하기 전에 학습이 끝날 때마다 출력을 지우도록 콜백 함수를 정의해봅시다.

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 첫 번째 Dense 노드 수는 {best_hps.get('units')} 입니다.
최적의 학습 속도는 {best_hps.get('learning_rate')} 입니다.
""")

Trial 30 Complete [00h 01m 22s]
val_accuracy: 0.8640999794006348

Best val_accuracy So Far: 0.886900007724762
Total elapsed time: 00h 17m 22s
INFO:tensorflow:Oracle triggered exit

하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 첫 번째 Dense 노드 수는 352 입니다.
최적의 학습 속도는 0.001 입니다.



7. **최고 성능을 보이는 하이퍼파라미터 조합으로 다시 학습을 진행해봅시다.**

In [ ]:
# 최적의 하이퍼 파라미터를 사용하여 모델을 구축하고 데이터에 대해 교육
model = tuner.hypermodel.build(best_hps)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 352)               276320    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3530      
Total params: 279,850
Trainable params: 279,850
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


1875/1875 [==============================] - 8s 4ms/step - loss: 0.4785 - accuracy: 0.8298 - val_loss: 0.4736 - val_accuracy: 0.8252
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3600 - accuracy: 0.8684 - val_loss: 0.3956 - val_accuracy: 0.8545
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3228 - accuracy: 0.8812 - val_loss: 0.3545 - val_accuracy: 0.8738
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2992 - accuracy: 0.8885 - val_loss: 0.3475 - val_accuracy: 0.8718
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2824 - accuracy: 0.8956 - val_loss: 0.3423 - val_accuracy: 0.8742
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2676 - accuracy: 0.8994 - val_loss: 0.3224 - val_accuracy: 0.8789
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2537 - accuracy: 0.9045 - val_loss: 0.3770 - val_accuracy: 0.8722
Epo

tuner에서 지정한 `my_dir/intro_to_kt` 경로에는 하이퍼파라미터 탐색 중에 실행되는 모든 모델에 대한 세부 로그와 체크포인트가 저장되어 있습니다.<br/>
동일한 모델로 하이퍼파라미터 탐색을 다시 실행할 때, Keras Tuner 기존 로그를 참고하여 검색을 시작합니다.<br/>
이 동작을 비활성화하려면 튜너 설정시 `overwrite = True` 로 지정해주어야 합니다.

> ❗️ ***Keras Tuner 를 더 알아보기 위한 학습 자료<br/>
1) [Keras Tuner 텐서플로우 블로그](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)<br/>
2) [Keras Tuner 공식문서](https://keras-team.github.io/keras-tuner/)<br/>
3) [HParams 대시보드 in Tensorboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) : 하이퍼파라미터 튜닝을 위한 대시보드 만들기***

### 실험 기록 툴(wandb 등)에 대해 알아보기


- **실험 기록을 하는 이유는 무엇일까요?**

다양한 하이퍼파라미터를 변경해가면서 장기적으로 실험을 진행하다보면 점점 결과를 관리하기가 어려워집니다.<br/>
"어떤 파라미터 조합이 제일 좋았지?", "어제 했던 결과와 차이가 있었던가?" 와 같은 의문을 품게 되죠.<br/>
비록 노트북(`.ipynb`)이 어느정도 출력물을 기록하기는 있지만 모든 실험 결과를 관리하기엔 적절하지 않습니다.

**Comet.ml, Weights and Biases(`wandb`)** 등은 이러한 문제를 해결하기 위해 등장한 실험 기록 도구입니다.<br/>
이런 실험 기록 도구는 실험 결과를 실시간으로 기록하고 **코드와 결과값을 보관**해주며,<br/>
실험 결과를 원하는 기준대로 언제든지 **시각화하여 모델의 성능을 확인**할 수 있도록 도와줍니다.<br/>
매 Epoch이 끝날 때마다 데이터가 해당 툴에 보내지기 때문에 **모델이 수렴하고 있는지도 확인**할 수 있습니다.

이번 시간에는 Weights and Biases(`wandb`)를 활용하여 실험 기록을 수행해보도록 하겠습니다.

#### Wandb 이용하기

- 설치 및 회원가입<br/>
먼저 다음 셀을 실행하기 전에 터미널에서 `wandb`에 로그인이 되있어야 합니다. 

    ```zsh
# 아래의 커맨드를 실행합니다
wandb.login
```
구체적인 방법은 Weights and Biases의 [QuickStart](https://docs.wandb.com/quickstart)를 참고해주시면 되겠습니다.

In [ ]:
!pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback

In [ ]:
# group, project 변수를 설정합니다. 반복되는 이름이 많기 때문에 변수로 설정하여 사용하면 편리합니다.
wandb_project = "review"
wandb_group = ""

In [ ]:
# !git clone http://github.com/wandb/tutorial

In [ ]:
# !cd tutorial; pip install --upgrade -r requirements.txt;

In [ ]:
!wandb login 6a1f7dd199ef2c241cc2dafb7ae52925d6de7385

In [ ]:
import numpy
import pandas as pd
#from tensorflow import keras
#from tensorflow.python import keras
#from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV

In [ ]:
# !python -c "import keras; print(keras.__version__)"

In [ ]:
wandb.init(project=wandb_project)  ## 내가 만든 프로젝트 이름을 넣어주어야 합니다.
#wandb.init(project=wandb_project, entity=wand_group) 

# 데이터 및 하이퍼파라미터 설정 
X =  x_train
y =  y_train

inputs = X.shape[1]
wandb.config.epochs = 50
wandb.config.batch_size = 10

# 모델을 구축합니다
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
# 모델을 컴파일 합니다
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# 모델을 학습합니다
model.fit(X, y, 
          validation_split=0.3, 
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
         )

In [ ]:
wandb.init(project=wandb_project)  ## 내가 만든 프로젝트 이름을 넣어주어야 합니다.

# 데이터 및 하이퍼파라미터 설정 
from tensorflow.keras import datasets
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

wandb.config.epochs = 10
wandb.config.batch_size = 64

# 모델을 구축합니다
model = Sequential() ## 과제시에는 이 모델을 Tre-trained model로 대체하면 됩니다. 
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()


In [ ]:
# 모델학습방식을 정의함
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습시키기
model.fit(train_images, train_labels, 
          validation_data=(test_images, test_labels),
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
          )

성능이 마음에 안든다면 추가로 학습을 더 시키는 방법도 있습니다. 

In [ ]:
wandb.config.epochs = 20
wandb.config.batch_size = 512

model.fit(train_images, train_labels, 
          validation_data=(test_images, test_labels),
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
          )

In [ ]:
!ls wandb/

다음으로는 프로그램 웹 페이지로 접속해서 분석해보는 시간을 가집니다.

## 🧐  Review

- 각 키워드에 대해서 간략하게 설명할 수 있는지 확인해 봅시다.
    - Activation Functions(활성화 함수)
    - Optimizer(옵티마이저)
    - Number of Layers
    - Number of Neurons
    - Batch Size(배치 사이즈)
    - Dropout(드롭아웃)
    - Learning Rate(학습률)
    - Number of Epochs
    - and many more
- Scikit-learn 과 Keras Tuner 를 통해서 여러분이 구축한 신경망에 여러 가지 하이퍼파라미터 조정 방법을 적용할 수 있는지 확인해봅니다.

### 🔝 References

- [Grid Search Hyperparameters for Deep Learning](https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/)
- [Hyperparameters Optimization for Deep Learning Models](https://blog.floydhub.com/guide-to-hyperparameters-search-for-deep-learning-models/)
- [Dropout Regularization in Deep Learning](https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/)
- [Weight Constraints in Deep Learning](https://machinelearningmastery.com/introduction-to-weight-constraints-to-reduce-generalization-error-in-deep-learning/)
- [Number of Layers and Nodes in a Neural Network](https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/)
- [Batch Normalization](https://shuuki4.wordpress.com/2016/01/13/batch-normalization-%EC%84%A4%EB%AA%85-%EB%B0%8F-%EA%B5%AC%ED%98%84/)

